In [26]:
%%capture
!pip install spotipy
!pip install python-louvain

import networkx as nx
import pandas as pd
import spotipy
from spotipy . oauth2 import SpotifyClientCredentials
import matplotlib.pyplot as plt
import operator
from collections import Counter
import community as community_louvain
import itertools
import numpy as np

In [7]:
CLIENT_ID = "fe233373eb024c3a97f1eeca78dba3c8"
CLIENT_SECRET = "34f0386560eb4dcb88e317f81b11b961"

auth_manager = SpotifyClientCredentials (client_id = CLIENT_ID, client_secret = CLIENT_SECRET)
sp = spotipy . Spotify ( auth_manager = auth_manager )

In [27]:
def num_common_nodes(*arg):
    """
    Return the number of common nodes between a set of graphs.

    :param arg: (an undetermined number of) networkx graphs.
    :return: an integer, number of common nodes.
    """
    common_nodes = set(arg[0].nodes())
    for g in arg[1:]:
        common_nodes.intersection_update(set(g.nodes()))
    return len(common_nodes)

def get_degree_distribution(g: nx.Graph) -> dict:
    """
    Get the degree distribution of the graph.

    :param g: networkx graph.
    :return: dictionary with degree distribution (keys are degrees, values are number of occurrences).
    """
    degrees = [g.degree(n) for n in g.nodes()]
    degree_distribution = Counter(degrees)
    return degree_distribution

def get_k_most_central(g: nx.Graph, metric: str, num_nodes: int) -> list:
    """
    Get the k most central nodes in the graph.

    :param g: networkx graph.
    :param metric: centrality metric. Can be (at least) 'degree', 'betweenness', 'closeness' or 'eigenvector'.
    :param num_nodes: number of nodes to return.
    :return: list with the top num_nodes nodes with the specified centrality.
    """
    if metric == 'degree':
        centrality = nx.degree_centrality(g)
    elif metric == 'betweenness':
        centrality = nx.betweenness_centrality(g)
    elif metric == 'closeness':
        centrality = nx.closeness_centrality(g)
    elif metric == 'eigenvector':
        centrality = nx.eigenvector_centrality(g)
    else:
        raise ValueError(f"Unknown metric: {metric}")

    sorted_centrality = sorted(centrality.items(), key=operator.itemgetter(1), reverse=True)
    return [node for node, centrality in sorted_centrality[:num_nodes]]

def find_cliques(g: nx.Graph, min_size_clique: int) -> tuple:
    """
    Find cliques in the graph g with size at least min_size_clique.

    :param g: networkx graph.
    :param min_size_clique: minimum size of the cliques to find.
    :return: two-element tuple, list of cliques (each clique is a list of nodes) and
        list of nodes in any of the cliques.
    """
    cliques = list(nx.find_cliques(g))
    cliques = [c for c in cliques if len(c) >= min_size_clique]
    nodes_in_cliques = set(itertools.chain.from_iterable(cliques))
    return cliques, list(nodes_in_cliques)

def detect_communities(g: nx.Graph, method: str) -> tuple:
    """
    Detect communities in the graph g using the specified method.

    :param g: a networkx graph.
    :param method: string with the name of the method to use. Can be (at least) 'givarn-newman' or 'louvain'.
    :return: two-element tuple, list of communities (each community is a list of nodes) and modularity of the partition.
    """
    if method == 'girvan_newman':
        communities_generator = nx.algorithms.community.girvan_newman(g)
        top_level_communities = next(communities_generator)
        communities = [list(c) for c in top_level_communities]
        modularity = nx.algorithms.community.modularity(g, communities)
    elif method == 'louvain':
        partition = community_louvain.best_partition(g)
        communities = list(set(partition.values()))
        communities = [[nodes for nodes, community in partition.items() if community == com] for com in communities]
        modularity = community_louvain.modularity(partition, g)
    else:
        raise ValueError(f"Unknown method: {method}")
    return communities, modularity
def retrieve_bidirectional_edges(g: nx.DiGraph, out_filename: str) -> nx.Graph:
    """
    Convert a directed graph into an undirected graph by considering bidirectional edges only.

    :param g: a networkx digraph.
    :param out_filename: name of the file that will be saved.
    :return: a networkx undirected graph.
    """
    # ------- IMPLEMENT HERE THE BODY OF THE FUNCTION ------- #
    out = nx.Graph()
    out.add_nodes_from(g)

    for edge in g.edges():  
      if edge[::-1] in g.edges(): #check if reverse edge exist in graph
        out.add_edge(edge[0],edge[1])
    
    out.remove_nodes_from(list(nx.isolates(out))) #Remove nodes without edges

    nx.write_graphml(out, out_filename+".graphml")
    return out
    # ----------------- END OF FUNCTION --------------------- #
    
def create_similarity_graph(artist_audio_features_df: pd.DataFrame, similarity: str, out_filename: str = None) -> nx.Graph:
    """
    Create a similarity graph from a dataframe with mean audio features per artist.

    :param artist_audio_features_df: dataframe with mean audio features per artist.
    :param similarity: the name of the similarity metric to use (e.g. "cosine" or "euclidean").
    :param out_filename: name of the file that will be saved.
    :return: a networkx graph with the similarity between artists as edge weights.
    """
    complete = nx.Graph()

    for index, row in artist_audio_features_df.iterrows():
      complete.add_node(index, features=row.tolist())  # index is considered as the name/id

    complete.add_edges_from(itertools.combinations(complete, 2))

    for ins,out,weight in complete.edges(data=True):
      features_in = np.array(dict(complete.nodes())[ins]['features'])  
      features_out = np.array(dict(complete.nodes())[out]['features']) 

      if similarity=='euclidean':
        dist = np.linalg.norm(features_in-features_out)
      elif similarity=='cosine':
        dist = dot(features_in, features_out)/(norm(features_in)*norm(features_out))
      
      weight['weight'] = dist 
    
    for node in list(complete.nodes()):
      del dict(complete.nodes())[node]['features']

    if out_filename is not None:
        nx.write_graphml(complete, out_filename+".graphml")

    return complete  


In [18]:
gB = nx.read_graphml('gB.graphml')
gD = nx.read_graphml('gD.graphml')
fB = nx.read_graphml('fB.graphml')
hB = nx.read_graphml('hB.graphml')

In [19]:
#Exercise 1
gB_prime = retrieve_bidirectional_edges(gB, 'gB_bi')
gD_prime = retrieve_bidirectional_edges(gD, 'gD_bi')

In [34]:
# Number of common nodes between gB and fB
common_nodes_gB_fB = num_common_nodes(gB, fB)
print("Number of common nodes between gB and fB:", common_nodes_gB_fB)

# Number of common nodes between gB and hB
common_nodes_gB_hB = num_common_nodes(gB, hB)
print("Number of common nodes between gB and hB:", common_nodes_gB_hB)




Number of common nodes between gB and fB: 27
Number of common nodes between gB and hB: 510


In [ ]:
# Creating similarity graphs

#!!!!!!!! needs revision
similarity_gB_fB = create_similarity_graph(gB_df, 'cosine', 'gB_fB')
similarity_gB_hB = create_similarity_graph(hB_df, 'cosine', 'gB_hB')


# Comparing the number of common nodes with similarity graph results
print("Number of nodes in the similarity graph (gB, fB):", similarity_gB_fB.number_of_nodes())
print("Number of nodes in the similarity graph (gB, hB):", similarity_gB_hB.number_of_nodes())

In [35]:
#Question 2
# Calculate the 25 most central nodes based on degree centrality
degree_central_nodes = get_k_most_central(gB_prime, 'degree', 25)

# Calculate the 25 most central nodes based on betweenness centrality
betweenness_central_nodes = get_k_most_central(gB_prime, 'betweenness', 25)

# Determine the number of nodes in common between the two sets
common_nodes = set(degree_central_nodes) & set(betweenness_central_nodes)
num_common_nodes = len(common_nodes)

print("Number of nodes in common between degree centrality and betweenness centrality:", num_common_nodes)



Number of nodes in common between degree centrality and betweenness centrality: 2


In [36]:
#Question 3
#Find cliques in gB' and gD' with the maximum value of min_size_clique that generates at least 2 cliques
max_possible_size_gB = len(gB_prime.nodes())
max_possible_size_gD = len(gD_prime.nodes())

min_size_clique_gB = None
min_size_clique_gD = None

for size in range(max_possible_size_gB, 1, -1):
    cliques_gB_prime, _ = find_cliques(gB_prime, size)
    if len(cliques_gB_prime) >= 2:
        min_size_clique_gB = size
        break

for size in range(max_possible_size_gD, 1, -1):
    cliques_gD_prime, _ = find_cliques(gD_prime, size)
    if len(cliques_gD_prime) >= 2:
        min_size_clique_gD = size
        break

# Step 5: Calculate the total number of cliques and total number of different nodes in all cliques for gB' and gD'
num_cliques_gB_prime = len(cliques_gB_prime)
num_nodes_in_cliques_gB_prime = len(set(node for clique in cliques_gB_prime for node in clique))

num_cliques_gD_prime = len(cliques_gD_prime)
num_nodes_in_cliques_gD_prime = len(set(node for clique in cliques_gD_prime for node in clique))

# Print the results
print("25 Most Central Nodes (Degree Centrality) in gB':", degree_central_nodes)
print("25 Most Central Nodes (Betweenness Centrality) in gB':", betweenness_central_nodes)
print("Number of Nodes in Common:", num_common_nodes)

print("Minimum Size of Cliques in gB':", min_size_clique_gB)
print("Number of Cliques in gB':", num_cliques_gB_prime)
print("Number of Different Nodes in All Cliques in gB':", num_nodes_in_cliques_gB_prime)

print("Minimum Size of Cliques in gD':", min_size_clique_gD)
print("Number of Cliques in gD':", num_cliques_gD_prime)
print("Number of Different Nodes in All Cliques in gD':", num_nodes_in_cliques_gD_prime)

25 Most Central Nodes (Degree Centrality) in gB': ['Huncho Jack', 'Yung Joc', 'Young Thug', 'Ab-Soul', 'Young Dro', 'Gorilla Zoe', 'Lil Scrappy', 'Boyz N Da Hood', 'Freddie Gibbs', 'Vince Staples', 'Quality Control', 'David Banner', 'Rich Boy', 'NAV', 'Famous Dex', 'Flipp Dinero', 'Rocko', 'Hurricane Chris', 'U.S.D.A.', 'Shawty Lo', 'PARTYNEXTDOOR', 'Gunna', 'Migos', 'A$AP Ferg', 'Jim Jones']
25 Most Central Nodes (Betweenness Centrality) in gB': ['DJ Drama', 'Curren$y', 'A$AP Twelvyy', 'A$AP Mob', 'Jeremih', 'Kid Ink', 'Desiigner', 'Flatbush Zombies', 'Yo Gotti', 'Bobby Shmurda', 'PARTYNEXTDOOR', 'O.T. Genasis', 'K CAMP', 'Swizz Beatz', 'DeJ Loaf', 'Waka Flocka Flame', 'Max B', 'French Montana', 'Mike WiLL Made-It', 'Trinidad James', 'Jacquees', 'Huncho Jack', 'Domo Genesis', 'Eric Bellinger', 'Young Money']
Number of Nodes in Common: 2
Minimum Size of Cliques in gB': 7
Number of Cliques in gB': 7
Number of Different Nodes in All Cliques in gB': 20
Minimum Size of Cliques in gD': 9
Nu

In [53]:
#Question 4
D = pd.read_csv('data_session-1.csv')
# Find the largest clique in gB' or gD' (depending on your choice)
largest_clique = max(nx.find_cliques(gB_prime), key=len)

print(largest_clique)
# Filter the artist DataFrame D to include only artists in the largest clique
clique_artists = D[D['artist_name'].isin(largest_clique)]

#print(clique_artists)
# Analyze the characteristics of the artists in the clique
common = clique_artists['explicit'].value_counts().head(5)

# Print the results
print("Artists in the largest clique:")
print(clique_artists['artist_name'])
print()
print("How common is theexplicit music among the artists:")
print(common)



['Yung Joc', 'Lil Scrappy', 'Boyz N Da Hood', 'Young Dro', 'Rich Boy', 'Shawty Lo', 'U.S.D.A.']
Artists in the largest clique:
999       Young Dro
1000      Young Dro
1001      Young Dro
1002      Young Dro
1003      Young Dro
           ...     
8566    Lil Scrappy
8567    Lil Scrappy
8568    Lil Scrappy
8569    Lil Scrappy
8570    Lil Scrappy
Name: artist_name, Length: 140, dtype: object

How common is theexplicit music among the artists:
explicit
True     126
False     14
Name: count, dtype: int64


In [42]:
#Question 5
communities, modularity = detect_communities(gD_prime, 'louvain') # or 'girvan_newman'


In [43]:
#Question 7
nx.shortest_path(gB_prime, 'Young Dro', 'Travis Porter')

['Young Dro',
 'Rocko',
 'Yo Gotti',
 'Young Money',
 'Soulja Boy',
 'Travis Porter']